<a href="https://colab.research.google.com/github/KaueAbbe/ChallengeDataScience2/blob/main/Tratamento_Dados_Semana1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [1]:
import os
os.environ['SPARK_HOME']= 'C:\spark\spark-3.3.1-bin-hadoop2'

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrameReader
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType, ArrayType

In [5]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [6]:
spark

Documentação pyspark
https://spark.apache.org/powered-by.html

# Lendo e Explorando a Base de Dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path ='/content/drive/MyDrive/PC novo/DS2/dataset_bruto.json'
df = spark.read.json(path)

In [8]:
print(f'São {df.count()} linhas e {len(df.columns)} colunas no dataframe')

São 89083 linhas e 3 colunas no dataframe


Com 89083 linhas e 3 colunas vou expandir a coluna anuncio. Nela deveremos obter mais colunas para serem trabalhadas posteriormente. As colunas são mostradas no Schema. 

# Expandindo o Anuncio

In [9]:
anuncio = df.select('anuncio.*')
anuncio.show(1)

+-----+----------+---------+---------+---------------+--------------------+--------------------+-------+------+------------+------------+---------+----+--------------------+
|andar|area_total|area_util|banheiros|caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade| tipo_uso|vaga|             valores|
+-----+----------+---------+---------+---------------+--------------------+--------------------+-------+------+------------+------------+---------+----+--------------------+
|    0|        []|     [16]|      [0]|             []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|Comercial| [1]|[{260, 107, Venda...|
+-----+----------+---------+---------+---------------+--------------------+--------------------+-------+------+------------+------------+---------+----+--------------------+
only showing top 1 row



In [10]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

# Filtrando colunas tipo: uso, unidade e anuncio

In [11]:
anuncio.groupBy(['tipo_uso', 'tipo_unidade', 'tipo_anuncio']).count().orderBy('count', ascending= False).show()

+-----------+------------+------------+-----+
|   tipo_uso|tipo_unidade|tipo_anuncio|count|
+-----------+------------+------------+-----+
|Residencial| Apartamento|       Usado|66562|
|Residencial|        Casa|       Usado|10224|
|Residencial|      Outros|       Usado| 7502|
|  Comercial|      Outros|       Usado| 4443|
|Residencial| Apartamento|  Lançamento|  235|
|  Comercial|        Casa|       Usado|   92|
|Residencial|      Outros|  Lançamento|   15|
|  Comercial| Apartamento|       Usado|    4|
|  Comercial|      Outros|  Lançamento|    3|
|Residencial|        Casa|  Lançamento|    3|
+-----------+------------+------------+-----+



In [12]:
anuncio.where('tipo_uso == "Residencial" and tipo_unidade == "Apartamento" and tipo_anuncio == "Usado"').show(1)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
only showing top 1 row



In [13]:
anuncio_filtrado = anuncio.where('tipo_uso == "Residencial" and tipo_unidade == "Apartamento" and tipo_anuncio == "Usado"')
anuncio_filtrado.show(1)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
only showing top 1 row



In [14]:
print(f'São {anuncio_filtrado.count()} linhas e {len(anuncio_filtrado.columns)} colunas no dataframe')

São 66562 linhas e 14 colunas no dataframe


Passamos a ter uma outra base de dados dos anúncios apenas para apartamentos usados com uso residencial. Assim temos um foco para o grupo alvo. 

A base de dados do anúncio contém 66562 linhas e 14 colunas.

# Transformando valores tipo listas em inteiros

In [15]:
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as f

In [16]:
anuncio_filtrado = anuncio_filtrado.withColumn('quartos', anuncio['quartos'][0].cast(IntegerType()))

In [17]:
anuncio_filtrado.select('quartos').show()

+-------+
|quartos|
+-------+
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
+-------+
only showing top 20 rows



In [18]:
for i in ['banheiros','suites','vaga','area_total','area_util']:
    anuncio_filtrado = anuncio_filtrado.withColumn(i, anuncio[i][0].cast(IntegerType()))

# Alterando colunas endereço e filtrando por bairro e zona

In [19]:
anuncio_filtrado.select('endereco.bairro').show(2)

+---------+
|   bairro|
+---------+
|Paciência|
|Paciência|
+---------+
only showing top 2 rows



In [20]:
anuncio_filtrado = anuncio_filtrado.join(anuncio_filtrado.select('endereco.bairro'))
anuncio_filtrado = anuncio_filtrado.join(anuncio_filtrado.select('endereco.zona'))


In [21]:
anuncio_filtrado = anuncio_filtrado.drop('endereco')

# Tratando valores

In [22]:
anuncio_filtrado.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)



In [23]:
for i in ['condominio','iptu','valor']:
    anuncio_filtrado = anuncio_filtrado.withColumn(i, anuncio_filtrado['valores'][i][0].cast(IntegerType()))
anuncio_filtrado.select('condominio')

DataFrame[condominio: int]

In [24]:
anuncio_filtrado = anuncio_filtrado.withColumn('tipo', anuncio_filtrado['valores']['tipo'][0])

In [25]:
anuncio_filtrado = anuncio_filtrado.drop('valores')

# Filtrando apenas por vendas

In [26]:
anuncio_filtrado = anuncio_filtrado.filter("tipo == 'Venda'")

A base de dados do anúncio contém 66562 linhas e 14 colunas.
O anuncio filtrado, agora é uma base de dados com de imóveis a venda para uso residencial. São apartamentos usados apenas.
A base foi tratada para ter tipos de valores corretos, e agora pode ser utilizada para análise exploratória do time de ciência de dados.


# Salvando Dados

In [27]:
df.write.parquet('dataset_tratado')